In [5]:
import math
import random
import copy

# Função para obter a representação computacional do tabuleiro aleatoriamente

In [45]:
def getRandomBoard(dimension:int):
    """Retorna um array que representa o tabuleiro (dimension x dimension) no qual cada coluna tem uma rainha posicionada.

    Representação Array 8x8: [2, 0, 4, 1, 6, 7, 3, 5] 

    Representação tabuleiro:

      0 . R . . . . . .
      1 . . . R . . . .
      2 R . . . . . . .
      3 . . . . . . R .
      4 . . R . . . . .
      5 . . . . . . . R
      6 . . . . R . . .
      7 . . . . . R . .
        0 1 2 3 4 5 6 7
    """
    board = [random.randint(0, 7) for _ in range(8)]
    # board = random.sample(range(0, dimension), k=dimension)
    return board
    

# Classe para definir o estado do tabuleiro

In [7]:
class BoardState:
    """ Define o estado. Cada coluna tem apenas uma rainha em cada estado."""

    def __init__(self, board:list) -> None:
        self.board = board
        self.dimension = len(self.board)
        self.horizontalConflictDict = {}
        self.mainDiagonalsConflictDict = {}
        self.secondaryDiagonalsConflictDict = {}
        self.heuristic = 0
        self.setConflictDictionaries()
        self.setHeuristic()
    
    
    def setConflictDictionaries(self) -> None:
        """Define 3 dicionários, que representam o número de rainhas:
            em cada linha -> horizontalConflictDict; 
            cada diagonal principal -> mainDiagonalsConflictDict;
            e cada diagonal secundária -> secondaryDiagonalsConflictDict
        Ex.: secondaryDiagonalsConflictDict[i] = k indica que existem k rainhas na i-ésima linha
        Obs.: 
            Diagonais Principais: a diferença entre os índices das linhas e das colunas é sempre igual na mesma diagonal
            Diagonais secundárias: a soma dos índices das linhas e das colunas é sempre igual na mesma diagonal
        """
        for j, i in enumerate(self.board):
                self.horizontalConflictDict[i] = self.horizontalConflictDict[i] + 1 if i in self.horizontalConflictDict else 1
                self.mainDiagonalsConflictDict[i-j] =  self.mainDiagonalsConflictDict[i-j] + 1 if i-j in self.mainDiagonalsConflictDict else 1
                self.secondaryDiagonalsConflictDict[i+j] =  self.secondaryDiagonalsConflictDict[i+j] + 1 if i+j in self.secondaryDiagonalsConflictDict else 1


    def setHeuristic(self) -> None:
        """Define a heurística de um estado. Essa heurística é o número de pares de rainhas que estão atacando uma a outra (Combinaçào de n elementos tomados 2 a 2)"""
        for boardLineIndex in self.horizontalConflictDict:
             if self.horizontalConflictDict[boardLineIndex] > 1:
                  self.heuristic += math.comb(self.horizontalConflictDict[boardLineIndex], 2)

        for boardMainDiagonalIndex in self.mainDiagonalsConflictDict:
             if self.mainDiagonalsConflictDict[boardMainDiagonalIndex] > 1:
                  self.heuristic += math.comb(self.mainDiagonalsConflictDict[boardMainDiagonalIndex], 2)

        for boardSecondaryDiagonalIndex in self.secondaryDiagonalsConflictDict:
             if self.secondaryDiagonalsConflictDict[boardSecondaryDiagonalIndex] > 1:
                  self.heuristic += math.comb(self.secondaryDiagonalsConflictDict[boardSecondaryDiagonalIndex], 2)


    def getRandomSuccessor(self) -> 'BoardState':
        """retorna um sucessor aleatório do estado atual"""
        j = random.randrange(0, self.dimension)
        possiblePositions = [i for i in range(self.dimension) if i != self.board[j]]
        i = random.choice(possiblePositions)

        newBoard = copy.deepcopy(self.board)
        newBoard[j] = i
        return BoardState(newBoard)
    

# Função que executa o Simulated Annealing baseado no estado inicial do tabuleiro

In [169]:
def runSimulatedAnnealing(boardState: BoardState, temperature:int=80000, temperatureDecay:int=0.8, maxNumImprovement:int=500) -> tuple:
    """Obtém o estado inicial do tabuleiro e executa o algoritmo Simulated Annealling"""
    currentState = BoardState(boardState)
    numImprovementCount = 0
    print("Iniciando Simulated Annealing")
    print("Estado atual do tabuleiro:")
    print(currentState.board)
    print(f"Heurística do estado atual: {currentState.heuristic}")
    print(f"{'='*50}")
    while True:
        temperature *= temperatureDecay
        if temperature < 1 or numImprovementCount >= maxNumImprovement:
            print("Estado final do tabuleiro: ")
            print(currentState.board)
            print(f"Heurística do estado final: {currentState.heuristic}")
            print(f"temperatura: {temperature}")
            print(f"contador de melhorias: {numImprovementCount}")
            if currentState.heuristic == 0:
                print("\n\033[1;32mO Simulated Annealing encontrou a solução ótima global\033[m")
                return True, currentState
            else:
                print("\n\033[1;31mO Simulated Annealing não conseguiu encontrar a solução ótima global\033[m")
                return False, currentState
            
        nextState = currentState.getRandomSuccessor()
        deltaE = nextState.heuristic - currentState.heuristic
        if deltaE < 0: #minimização
            print("Movimentação realizada:")
            print(f"{currentState.board} ---> {nextState.board}")
            print("Estado atual do tabuleiro:")
            print(currentState.board)
            print(f"Heurística do estado atual: {currentState.heuristic}")
            print(f"{'='*50}")
            currentState = nextState
            numImprovementCount = 0
        else:
            randomDecision = random.uniform(0, 1)
            probability = math.exp(deltaE/temperature)
            if randomDecision >= probability:
                print("Movimentação realizada:")
                print(f"{currentState.board} ---> {nextState.board}")
                print("Estado atual do tabuleiro:")
                print(currentState.board)
                print(f"Heurística do estado atual: {currentState.heuristic}")
                print(f"{'='*50}")
                currentState = nextState
                numImprovementCount = 0 # reseta o contador se a mudança for aceita
            else:
                numImprovementCount += 1 # incrementa o contador se não houver melhoria

# Execução

In [278]:
randomBoard = getRandomBoard(dimension=8)
solution = runSimulatedAnnealing(boardState=randomBoard)
print(f"\033[0;39mAchou solução ótima? {solution[0]}")
print(f"Estado final do tabuleiro: {solution[1].board}")
print(f"Heurística final: {solution[1].heuristic}")


Iniciando Simulated Annealing
Estado atual do tabuleiro:
[4, 0, 3, 1, 7, 3, 4, 3]
Heurística do estado atual: 9
Movimentação realizada:
[4, 0, 3, 1, 7, 3, 4, 3] ---> [4, 0, 3, 5, 7, 3, 4, 3]
Estado atual do tabuleiro:
[4, 0, 3, 1, 7, 3, 4, 3]
Heurística do estado atual: 9
Movimentação realizada:
[4, 0, 3, 5, 7, 3, 4, 3] ---> [4, 0, 3, 5, 7, 3, 4, 0]
Estado atual do tabuleiro:
[4, 0, 3, 5, 7, 3, 4, 3]
Heurística do estado atual: 7
Movimentação realizada:
[4, 0, 3, 5, 7, 3, 4, 0] ---> [4, 0, 3, 5, 7, 2, 4, 0]
Estado atual do tabuleiro:
[4, 0, 3, 5, 7, 3, 4, 0]
Heurística do estado atual: 5
Movimentação realizada:
[4, 0, 3, 5, 7, 2, 4, 0] ---> [4, 0, 3, 5, 7, 1, 4, 0]
Estado atual do tabuleiro:
[4, 0, 3, 5, 7, 2, 4, 0]
Heurística do estado atual: 3
Movimentação realizada:
[4, 0, 3, 5, 7, 1, 4, 0] ---> [4, 0, 3, 5, 7, 1, 6, 0]
Estado atual do tabuleiro:
[4, 0, 3, 5, 7, 1, 4, 0]
Heurística do estado atual: 2
Movimentação realizada:
[4, 0, 3, 5, 7, 1, 6, 0] ---> [4, 0, 3, 5, 7, 1, 6, 2]
Esta